# Import Libraries


In [7]:
import pandas as pd # data processing
import numpy as np # Mathematical operations
import seaborn as sns # Data visualization
import matplotlib.pyplot as plt # Data visualization
import moviepy.editor as mpy # Video and audio editing
import requests # HTTP requests
import os # File operations
import sys # System operations
import boto3 # AWS SDK
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pydub import AudioSegment # Audio processing

# Media Files

    A. Issues when loading the video: 

                1. When previewing the video it has a mixer issue - Will skip  this for now till fix is found, will convert the file and store to bucket
                2. Where do we store the temporary files - Files will be stored in a bucket on S3 on AWS

**File convertion**

In [ ]:
file_path = "C:/Users/tedjo/Downloads/dancevibedaily/media/"
save_path = "C:/Users/tedjo/Downloads/dancevibedaily/media/audio/"

#Loop through each file within the directory
for file in os.listdir(file_path):
    if file.endswith(".mp4"):
        video_path = os.path.join(file_path, file)

        #Load video file
        try:
            video = mpy.VideoFileClip(video_path)
            print(f'Video Loaded: {video_path}')
            audio = video.audio
            audio.write_audiofile(os.path.join(save_path,f'{file}.mp3'))
            
            # #Access AWS S3 Bucket
            # aws_access_key_id = ''
            # aws_secret_access_key = ''

            # s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

            # with open(os.path.join(file_path,f'{file}.mp3'), 'rb') as data:
            #     s3.Bucket('dancevibedaily').put_object(Key=f'{file}.mp3', Body=data)
            #     print(f'Audio file uploaded to S3: {file}.mp3')

            # #Remove audio file from local storage
            # os.remove(os.path.join(file_path,f'{file}.mp3'))
            # print("Audio file removed from local storage")

        except Exception as e:
            print(f"Error loading video: {e}")
            continue

# Data Building and Cleaning

**Issues on API use with bucket**

                1. As I am new to Data Engineering I have not figured out how to work with api for now will work with local directory.
                2. There is a 'NoneType' object is not subscriptable, will go through these json and get to understand the issue.
                3. Will need to rename the media file to match the song and data in the uri, to be able to identify track
                   1. To deal with this I have created a uri dataframe which will hold the mp3 file name and title of track (Done)
                   2. This will help in finding:
                      1. Tracks that may not have a match
                      2. Match the number of likes to the song 

**Audd API**

In [75]:
import requests # HTTPS Requests
data = {
    'api_token': '1f11b47922b072fe40304ce3cefd8c69',
    'return': 'musicbrainz',
}

# Will loop through the files in the directory and extract the song title, artist, and album from the API response
save_path = "C:/Users/tedjo/Downloads/dancevibedaily/media/audio/"
results = [] # List to store the results of the API response
uri_song = [] # Reel JSON URI and the Title of the song

for i in os.listdir(save_path):
    if i.endswith(".mp3"):
        files = {
            'file': open(os.path.join(save_path,i),'rb') #reading binary file
        }
        try:
            result = requests.post('https://api.audd.io/', data=data, files=files) # Send data to the API 
            if result.json()['status'] == 'error':
                print(f"Error processing file: {result.json()}")
                continue
            print(result.json()['result']['title'])
            results.append({
                'Title': result.json()['result']['title'],
                'Artist': result.json()['result']['artist'],
                'Album': result.json()['result']['album'],
                'Release Date': result.json()['result']['release_date'],
            })# Append the results to the list, which will be used to create a DataFrame
            uri_song.append({
                'uri': i,
                'Title': result.json()['result']['title'],
            })
        except Exception as e:
            print(f"Error processing file: {e}")
            continue

df = pd.DataFrame(results) # Create a DataFrame from the results list
df_1 = pd.DataFrame(uri_song)


All Over You
ULAZI Infinity MusiQ (yey)
Tjina
YESHUA (dance cruise)
Private school
Error processing file: 'NoneType' object is not subscriptable
Look at Her
Killin Dem
Error processing file: 'NoneType' object is not subscriptable
Escapism. (Sped Up)
Error processing file: 'NoneType' object is not subscriptable
Error processing file: 'NoneType' object is not subscriptable
Jinja
Dalie (feat. Baby S.O.N)
soso
الي اين انا ذاهب
Water
Issa Necessary (feat. Kiss Daniel)
Mnike (UK Radio Edit)
Barbie World (with Aqua) [From Barbie The Album]
UNAVAILABLE (feat. Musa Keys)
Look at Her
Hwiralang feat. Siko Wa Mmino,Hlogi Mash
dabala
Mnike
Cough
Error processing file: 'NoneType' object is not subscriptable
Water
Venda Boy (feat. fefe laku dura)
AH TXE TXE (feat. TYSON)
Floor Seats
Error processing file: 'NoneType' object is not subscriptable
Rock Attack
Walking dead
Kafe'
Shu! (feat. Chley)
Error processing file: 'NoneType' object is not subscriptable
Mnike
Get Up (Main Version)
Lost In The Bass. M

In [ ]:
df.to_csv('C:/Users/tedjo/Downloads/dancevibedaily/media/audio/audio_metadata.csv', index=False) # Save the DataFrame to a CSV file
df_1.to_csv('C:/Users/tedjo/Downloads/dancevibedaily/media/audio/uri_reel_data.csv', index=False)

In [8]:
df = pd.read_csv('C:/Users/tedjo/Downloads/dancevibedaily/media/audio/audio_metadata.csv') # Music names from videos
df_1 = pd.read_csv('C:/Users/tedjo/Downloads/dancevibedaily/media/audio/uri_reel_data.csv')# URI data which will be used to match the quantity data from reels

In [9]:
#Merge df and df_1
df = df.merge(df_1, on='Title', how='inner')
df.drop_duplicates(subset=['uri'], keep='first', inplace=True)
df['uri'] = df['uri'].apply(lambda x: x.split('_')[0])
df.reset_index(drop=True, inplace=True)
df.drop(columns = ['Release Date'], inplace=True)

**Spotify API**

In [10]:
client_id = 'b54b42aba265429c81a6e97d46fcfec2'# Spotify Client Token
client_secret = 'bb6c018ee46f40d595a04c9ab076794d'  # Spotify Secret Token
auth_manager = SpotifyClientCredentials(client_id = client_id, client_secret=client_secret)# Initialize the Spotify Client Credentials
sp = spotipy.Spotify(auth_manager=auth_manager)# Initialize the Spotify API client

# #New Columns
# df['Spotify Track ID'] ='' # Track ID for other analysis on tracks
# df['Spotify Track Name'] = ''# Track Name
# df['Sporify Album'] = '' # Album Name
# df['Spotify Arist'] = '' # Artists Name
# df['Spotify Track URL'] = '' # Track URL
# df['Spotify Track Genre'] = ''#Track Genre
# df['Spotify Image Path'] = ''#Image Path
# df['Spotify Track Popularity'] = ''#How popular is this track from 0 to 100

#Function to get details
def get_details(track_name,artist_name,album_name):
    query = f'track: {track_name} artist: {artist_name} album: {album_name}'
    result  = sp.search(q= query, type= 'track',limit=1 )
    if result['tracks']['items']:
        track = result['tracks']['items'][0]
        artist_id = track['artists'][0]['id']
        artist = sp.artist(artist_id)
        return{
            'Spotify Track ID': track['id'],
            'Spotify Track Name': track['name'],
            'Spotify Artist': ', '.join([artist['name'] for artist in track['artists']]),
            'Spotify Artist Image URL': artist['images'][0]['url'] if artist['images'] else None,
            'Spotify Album': track['album']['name'],
            'Spotify URL': track['external_urls']['spotify'],
            'Spotify Image URL': track['album']['images'][0]['url'] if track['album']['images'] else None,
            'Spotify Track Popularity': track['popularity'],
            'Spotify Track Genre': ', '.join(artist['genres'])  if artist['genres'] else None
        }
    else:
        return None

for index, row in df.iterrows():
    details = get_details(row['Title'],row['Artist'],row['Album'])
    print(f"Processing {index+1} of {len(df)}")
    print(details)
    if details:
        df.at[index, 'Spotify Track ID'] = details['Spotify Track ID']
        df.at[index, 'Spotify Track Name'] = details['Spotify Track Name']
        df.at[index, 'Spotify Artist'] = details['Spotify Artist']
        df.at[index, 'Spotify Artist Image URL'] = details['Spotify Artist Image URL']
        df.at[index, 'Spotify Album'] = details['Spotify Album']
        df.at[index, 'Spotify Track URL'] = details['Spotify URL']
        df.at[index, 'Spotify Image URL'] = details['Spotify Image URL']
        df.at[index, 'Spotify Track Popularity'] = details['Spotify Track Popularity']
        df.at[index, 'Spotify Track Genre'] = details['Spotify Track Genre']



Processing 1 of 857
{'Spotify Track ID': '75Q69chmd8CEZbVsA4CDMm', 'Spotify Track Name': 'Gorgeous', 'Spotify Artist': 'mansionz', 'Spotify Artist Image URL': 'https://i.scdn.co/image/ab6761610000e5ebbfa6fdfaedc557d0c669b3a1', 'Spotify Album': 'Mansionz', 'Spotify URL': 'https://open.spotify.com/track/75Q69chmd8CEZbVsA4CDMm', 'Spotify Image URL': 'https://i.scdn.co/image/ab67616d0000b27390d0e1eb630b1595d23a1a0e', 'Spotify Track Popularity': 56, 'Spotify Track Genre': 'indie pop rap'}
Processing 2 of 857
{'Spotify Track ID': '4k2yKSDE5cSUED46GFbSdd', 'Spotify Track Name': 'ULAZI Infinity MusiQ (yey)', 'Spotify Artist': 'uLazi', 'Spotify Artist Image URL': 'https://i.scdn.co/image/ab6761610000e5eb42c5d64bd48e304f8c78f3aa', 'Spotify Album': 'ULAZI Infinity MusiQ (yey)', 'Spotify URL': 'https://open.spotify.com/track/4k2yKSDE5cSUED46GFbSdd', 'Spotify Image URL': 'https://i.scdn.co/image/ab67616d0000b2739a1373a5bb0ce5db2c080155', 'Spotify Track Popularity': 32, 'Spotify Track Genre': 'sgija

**Spotify Audio Features**


                Danceability: Describes how suitable a track is for dancing. It is based on a combination of musical elements such as tempo, rhythm stability, beat strength, and overall regularity. A higher value indicates that the track is more danceable.

                Energy: A measure of intensity and activity. Tracks with high energy are usually fast, loud, and noisy. This attribute typically corresponds to fast, upbeat tracks, while low energy tracks are more likely to be relaxed or slow.

                Loudness: The overall loudness of a track in decibels (dB). Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Tracks with higher loudness values are typically perceived as louder.

                Mode: Indicates the modality (major or minor) of a track. Major is represented by a value of 1, while minor is represented by a value of 0. Mode is related to the scale of the track and can affect the mood it conveys.

                Speechiness: Reflects the presence of spoken words in a track. Values closer to 1.0 indicate tracks that are primarily speech (e.g., talk shows, podcasts). Values closer to 0.0 are more likely to be purely musical with little to no speech.

                Acousticness: A measure of how acoustic a track is. Higher acousticness values indicate that the track is more likely to be acoustic or organic in nature (e.g., live performances, acoustic instruments).

                Instrumentalness: Predicts whether a track contains no vocals. A higher value indicates that the track is more likely to be instrumental. Values closer to 1.0 suggest a higher likelihood of the track being purely instrumental.

                Liveness: Detects the presence of a live audience in the recording. Higher liveness values indicate that the track was recorded in a live setting, which may include audience noise and other live elements.

                Valence: A measure of the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g., happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g., sad, depressed, angry).

                Tempo: The speed or pace of a given track, measured in beats per minute (BPM). It reflects the overall tempo of the track and can influence the energy and mood of the music.

In [10]:
df.head(2)

,Title,Artist,Album,Release Date,uri,Spotify Track ID,Spotify Track Name,Spotify Artist,Spotify Artist Image URL,Spotify Album,Spotify Track URL,Spotify Image URL,Spotify Track Popularity,Spotify Track Genre,Danceability
0,All Over You,Sped Up 204,GORGEOUS,2023-09-08,034307877B906E78461292BF61ADEDBC_video_dashini...,75Q69chmd8CEZbVsA4CDMm,Gorgeous,mansionz,https://i.scdn.co/image/ab6761610000e5ebbfa6fd...,Mansionz,https://open.spotify.com/track/75Q69chmd8CEZbV...,https://i.scdn.co/image/ab67616d0000b27390d0e1...,56.0,indie pop rap,
1,ULAZI Infinity MusiQ (yey),uLazi,ULAZI Infinity MusiQ (yey),2022-12-01,034377C77CC9159980937FFB85869E91_video_dashini...,4k2yKSDE5cSUED46GFbSdd,ULAZI Infinity MusiQ (yey),uLazi,https://i.scdn.co/image/ab6761610000e5eb42c5d6...,ULAZI Infinity MusiQ (yey),https://open.spotify.com/track/4k2yKSDE5cSUED4...,https://i.scdn.co/image/ab67616d0000b2739a1373...,32.0,sgija,
2,ULAZI Infinity MusiQ (yey),uLazi,ULAZI Infinity MusiQ (yey),2022-12-01,FQIoAkMzLBdAKRDlYEGJNxgSZGFzaF9iYXNlbGluZV8xX3...,4k2yKSDE5cSUED46GFbSdd,ULAZI Infinity MusiQ (yey),uLazi,https://i.scdn.co/image/ab6761610000e5eb42c5d6...,ULAZI Infinity MusiQ (yey),https://open.spotify.com/track/4k2yKSDE5cSUED4...,https://i.scdn.co/image/ab67616d0000b2739a1373...,32.0,sgija,
3,ULAZI Infinity MusiQ (yey),uLazi,ULAZI Infinity MusiQ (yey),2022-12-01,FQIoAkMzLBdAPkQYk3S8ahgSZGFzaF9iYXNlbGluZV8xX3...,4k2yKSDE5cSUED46GFbSdd,ULAZI Infinity MusiQ (yey),uLazi,https://i.scdn.co/image/ab6761610000e5eb42c5d6...,ULAZI Infinity MusiQ (yey),https://open.spotify.com/track/4k2yKSDE5cSUED4...,https://i.scdn.co/image/ab67616d0000b2739a1373...,32.0,sgija,
4,ULAZI Infinity MusiQ (yey),uLazi,ULAZI Infinity MusiQ (yey),2022-12-01,FQIoAkMzLBdAPyIMSbpeNRgSZGFzaF9iYXNlbGluZV8xX3...,4k2yKSDE5cSUED46GFbSdd,ULAZI Infinity MusiQ (yey),uLazi,https://i.scdn.co/image/ab6761610000e5eb42c5d6...,ULAZI Infinity MusiQ (yey),https://open.spotify.com/track/4k2yKSDE5cSUED4...,https://i.scdn.co/image/ab67616d0000b2739a1373...,32.0,sgija,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5588,Feeling for You,Reefer Tym,Feeling for You,2022-04-08,pxBFQIoAkMzLBdANLMzMzMzMxgSZGFzaF9iYXNlbGluZV8...,5I75BCLNYetvV57NM6KneG,Feeling for You,Reefer Tym,https://i.scdn.co/image/ab6761610000e5ebd2b580...,Feeling for You,https://open.spotify.com/track/5I75BCLNYetvV57...,https://i.scdn.co/image/ab67616d0000b273b107d1...,35.0,None,
5657,"Howa You feat. Myztro,Xduppy",Shaunmusiq/Ftears/Daliwonga,"Howa You feat. Myztro,Xduppy",2023-08-11,Ta7tJAFQIoAkMzLBdAS0iTdLxqfxgSZGFzaF9iYXNlbGlu...,57T2jbt9nqR6VCeOH72wuG,Howa You (feat. Myztro & Xduppy),"ShaunMusiq & Ftears, Daliwonga, Myztro, Xduppy",https://i.scdn.co/image/ab6761610000e5eb8ac77f...,Howa You (feat. Myztro & Xduppy),https://open.spotify.com/track/57T2jbt9nqR6VCe...,https://i.scdn.co/image/ab67616d0000b27332bdd7...,42.0,"amapiano, quantum sound",
5658,Skeletal Landscape,"Tanesha Kasongo,Zemora",Skeletal Landscape,2020-07-30,u6pAFQIoAkMzLBdATN3ztkWhyxgSZGFzaF9iYXNlbGluZV...,2VNRWCCSjXSkmLZkJ4x40h,Landscape,Deluxe,https://i.scdn.co/image/ab6761610000e5ebbe2ab4...,Landscape,https://open.spotify.com/track/2VNRWCCSjXSkmLZ...,https://i.scdn.co/image/ab67616d0000b273d02f3d...,40.0,None,
5661,Remake,ENNYMAN DA GUITAR,Remake,2023-06-02,vmp3Y5AFQIoAkMzLBdAR6qfvnbItBgSZGFzaF9iYXNlbGl...,055YvxM3Ljr94gf1Zh4qKR,Still Alive,Enny Man Da Guitar,https://i.scdn.co/image/ab6761610000e5eb90c6c7...,The Legacy EP: The Rise Of A Legend,https://open.spotify.com/track/055YvxM3Ljr94gf...,https://i.scdn.co/image/ab67616d0000b273fc92b9...,20.0,barcadi,


In [11]:
df['uri'] = df['uri'].apply(lambda x: x.split('_')[0])

In [28]:
df

,Title,Artist,Album,uri
0,All Over You,Sped Up 204,GORGEOUS,034307877B906E78461292BF61ADEDBC
1,ULAZI Infinity MusiQ (yey),uLazi,ULAZI Infinity MusiQ (yey),034377C77CC9159980937FFB85869E91
2,ULAZI Infinity MusiQ (yey),uLazi,ULAZI Infinity MusiQ (yey),FQIoAkMzLBdAKRDlYEGJNxgSZGFzaF9iYXNlbGluZV8xX3...
3,ULAZI Infinity MusiQ (yey),uLazi,ULAZI Infinity MusiQ (yey),FQIoAkMzLBdAPkQYk3S8ahgSZGFzaF9iYXNlbGluZV8xX3...
4,ULAZI Infinity MusiQ (yey),uLazi,ULAZI Infinity MusiQ (yey),FQIoAkMzLBdAPyIMSbpeNRgSZGFzaF9iYXNlbGluZV8xX3...
...,...,...,...,...
852,Feeling for You,Reefer Tym,Feeling for You,pxBFQIoAkMzLBdANLMzMzMzMxgSZGFzaF9iYXNlbGluZV8...
853,"Howa You feat. Myztro,Xduppy",Shaunmusiq/Ftears/Daliwonga,"Howa You feat. Myztro,Xduppy",Ta7tJAFQIoAkMzLBdAS0iTdLxqfxgSZGFzaF9iYXNlbGlu...
854,Skeletal Landscape,"Tanesha Kasongo,Zemora",Skeletal Landscape,u6pAFQIoAkMzLBdATN3ztkWhyxgSZGFzaF9iYXNlbGluZV...
855,Remake,ENNYMAN DA GUITAR,Remake,vmp3Y5AFQIoAkMzLBdAR6qfvnbItBgSZGFzaF9iYXNlbGl...


In [12]:
import numpy as np

for index, row in df.iterrows():
    print(f"Processing {index+1} of {len(df)}")
    
    
    # # Check if 'Danceability' is NaN
    # if not pd.isna(row['Danceability']):
    #     continue  # Skip to the next row if 'Danceability' is already filled
    
    # If 'Danceability' is NaN and 'Spotify Track ID' exists, fetch the audio features
    if row['Spotify Track ID']:
        try:
            features = sp.audio_features(row['Spotify Track ID'])
            if features and features[0]:  # Check if the features list is not empty
                df.at[index, 'Danceability'] = features[0]['danceability']
                df.at[index, 'Energy'] = features[0]['energy']
                df.at[index, 'Loudness'] = features[0]['loudness']
                df.at[index, 'Mode'] = features[0]['mode']
                df.at[index, 'Speechiness'] = features[0]['speechiness']
                df.at[index, 'Acousticness'] = features[0]['acousticness']
                df.at[index, 'Instrumentalness'] = features[0]['instrumentalness']
                df.at[index, 'Liveness'] = features[0]['liveness']
                df.at[index, 'Valence'] = features[0]['valence']
                df.at[index, 'Tempo'] = features[0]['tempo']
            else:
                print(f"Failed to retrieve features for Spotify Track ID: {row['Spotify Track ID']}")
        except Exception as e:
            print(f"Error processing Spotify Track ID {row['Spotify Track ID']}: {e}")

Processing 1 of 857
Processing 2 of 857
Processing 3 of 857
Processing 4 of 857
Processing 5 of 857
Processing 6 of 857
Processing 7 of 857
Processing 8 of 857
Processing 9 of 857
Processing 10 of 857
Processing 11 of 857
Processing 12 of 857
Processing 13 of 857
Processing 14 of 857
Processing 15 of 857
Processing 16 of 857
Processing 17 of 857
Failed to retrieve features for Spotify Track ID: 6sir4HTyiafmPsQFaaYdXk
Processing 18 of 857
Processing 19 of 857
Processing 20 of 857
Processing 21 of 857
Processing 22 of 857
Processing 23 of 857
Processing 24 of 857
Processing 25 of 857
Processing 26 of 857
Processing 27 of 857
Processing 28 of 857
Processing 29 of 857
Processing 30 of 857
Processing 31 of 857
Processing 32 of 857
Processing 33 of 857
Processing 34 of 857
Processing 35 of 857
Processing 36 of 857
Processing 37 of 857
Processing 38 of 857
Processing 39 of 857
Processing 40 of 857
Processing 41 of 857
Processing 42 of 857
Processing 43 of 857
Processing 44 of 857
Processing 4

# Data Cleaning

In [13]:
import json 

import pandas as pd 
reels= 'C:/Users/tedjo/OneDrive/Documents/Beyond Words/Projects/Dance Daily/Data/reels.json'

with open(reels) as f:
    data = json.load(f)

data



{'organic_insights_reels': [{'title': '',
   'media_map_data': {'Media thumbnail': {'uri': 'media/other/FQIoAkMzLBdAMoDEm6XjVBgSZGFzaF9iYXNlbGluZV8xX3YxEQB1AAA_18007635686289180',
     'creation_timestamp': 1718740500,
     'media_metadata': {'video_metadata': {'exif_data': [{'device_id': 'BB74F8D8-24C3-479A-B209-37318D53BD8E',
         'camera_position': 'unknown',
         'source_type': '0'}]},
      'camera_metadata': {'has_camera_metadata': False}},
     'title': 'She slayed this dance ð\x9f\x94¥â\x9c¨\n\nBIO @dancevibesdaily\n\nDont forget to Like or Share ð\x9f\x99\x8fð\x9f\x8f¾\n________________________________________________\n\nâ\x99»ï¸\x8f: @laura.delvecchio87 \n________________________________________________\n#reels #reelsinstagram #viralreels #dance #afrofusion #afro #afrobeats  #dancereels #explorepage #explore \n#trendingsongs',
     'cross_post_source': {'source_app': 'FB'}}},
   'string_map_data': {'Caption': {'href': '',
     'value': 'She slayed this dance ð\x9f\x94

In [14]:
with open(reels) as f:
    data = json.load(f)


# Adjusting the extraction to handle cases where some keys might be missing
def extract_data(data):
    extracted_data = []
    for item in data['organic_insights_reels']:
        uri  = item['media_map_data'].get('Media thumbnail', {}).get('uri','N/A')
        uri = uri.split('/')[-1]
        uri = uri.split('_')[0]
        caption = item["string_map_data"].get("Caption", {}).get("value", "N/A")
        #In the captions there will be emojis need to process this
        caption = caption.encode('latin1').decode('utf8')
        #need to process \n
        caption = caption.replace('\n','')
        upload_timestamp = item["string_map_data"].get("Upload timestamp", {}).get("timestamp", "N/A")
        accounts_reached = item["string_map_data"].get("Accounts reached", {}).get("value", "N/A")
        plays = item["string_map_data"].get("Instagram plays", {}).get("value", "N/A")
        likes = item["string_map_data"].get("Instagram likes", {}).get("value", "N/A")
        comments = item["string_map_data"].get("Instagram comments", {}).get("value", "N/A")
        saves = item["string_map_data"].get("Instagram saves", {}).get("value", "N/A")
        duration = item["string_map_data"].get("Duration", {}).get("value", "N/A")
        facebook_play = item["string_map_data"].get("Facebook play count",{}).get("value","N/A")
        facebook_likes = item['string_map_data'].get('Facebook likes',{}).get('value','N/A')
        
        # Extract music genre if available
        music_genre = "N/A"
        media_metadata = item["media_map_data"]["Media thumbnail"].get("media_metadata", {})
        if "video_metadata" in media_metadata and "music_genre" in media_metadata["video_metadata"]:
            music_genre = media_metadata["video_metadata"]["music_genre"]
        
        
        extracted_data.append({
            'uri': uri,
            "caption": caption,
            "upload_timestamp": upload_timestamp,
            "accounts_reached": accounts_reached,
            "plays": plays,
            "likes": likes,
            "comments": comments,
            "saves": saves,
            "duration": duration,
            "facebook_play": facebook_play,
            "facebook_likes": facebook_likes
        })
    
    return extracted_data

extracted_data = extract_data(data)

# Create a DataFrame
metric = pd.DataFrame(extracted_data)

metric['upload_timestamp'] = pd.to_datetime(metric['upload_timestamp'], unit='s')

metric[metric['facebook_play'] != 'N/A']

# metric.to_csv('C:/Users/tedjo/Downloads/reels_data.csv', index=False)


,uri,caption,upload_timestamp,accounts_reached,plays,likes,comments,saves,duration,facebook_play,facebook_likes
18,FQIoAkMzLBdANuZmZmZmZhgSZGFzaF9iYXNlbGluZV8xX3...,When you been pregnant for way too long 🤰🏾BIO ...,2024-06-09 18:03:31,21034,24910,1535,15,143,23,1026,12
41,FQIoAkMzLBdAK41P3ztkWhgSZGFzaF9iYXNlbGluZV8xX3...,15 seconds of Tyla 🔥______________♻️: @tyla___...,2024-05-29 22:38:25,13108,15679,958,8,99,14,45,0
42,FQIoAkMzLBdAUEiDEm6XjRgSZGFzaF9iYXNlbGluZV8xX3...,Davido or Wizkid: Who won 🇳🇬🤺______________dan...,2024-05-29 20:52:41,7174,8348,349,4,49,66,8609,69
52,FQIoAkMzLBdATM2RaHKwIRgSZGFzaF9iYXNlbGluZV8xX3...,Barcadi to the world! Follow for more @dancevi...,2024-05-26 18:45:41,24627,28564,2071,20,269,58,83,0
54,DB4B4050F174279EC7DCE94AB9FBD087,Kelly got a new nickname ✨Follow for more @dan...,2024-05-26 10:58:22,11597,13165,618,10,63,41,34,0
...,...,...,...,...,...,...,...,...,...,...,...
1384,FQIoAkMzLBdALHdLxqfvnhgSZGFzaF9iYXNlbGluZV8xX3...,REPOST @mjemjaySong 🎵: @theonlykidda - low #__...,2023-02-22 16:48:01,2282,2427,11,2,3,15,0,0
1385,FQIoAkMzLBdATgAAAAAAABgSZGFzaF9iYXNlbGluZV8xX3...,REPOSTChorégraphie @badgyalcassiee 🎥 @amegavis...,2023-02-22 16:28:20,1168,2577,6,2,0,60,1,0
1386,FQIoAkMzLBdAP5mZmZmZmhgSZGFzaF9iYXNlbGluZV8xX3...,REPOST🕺🏾: @masterkidk 🧠: @bxujii cc: @chopdail...,2023-02-22 13:38:06,2036,2162,2,1,0,32,0,0
1387,FQIoAkMzLBdANl3ztkWhyxgSZGFzaF9iYXNlbGluZV8xX3...,REPOST @onedance_tv 💯🔥________________________...,2023-02-22 13:33:23,47,50,2,2,0,23,6,0


In [15]:
metric.to_csv('C:/Users/tedjo/Downloads/reels_data_cleaned.csv', index=False)
df.to_csv('C:/Users/tedjo/Downloads/dancevibedaily/media/audio/audio_metadata_filled.csv', index=False) # Save the DataFrame to a CSV file


# Sentiment Analysis on captions

**Overall**

In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()


In [36]:
metric['sentiment'] = metric['caption'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


In [39]:
metric.drop(columns = ['sentiment_category'], inplace=True)

In [52]:
def sentiment_category(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'
    
metric['sentiment_category'] = metric['sentiment'].apply(sentiment_category)

metric.groupby('sentiment_category')['likes'].sum()


sentiment_category
Negative     213331
Neutral      182065
Positive    3151785
Name: likes, dtype: int64

In [43]:
# Group by sentiment category and calculate mean engagement metrics
sentiment_vs_engagement = df.groupby('sentiment_category')[['plays', 'likes', 'comments', 'saves', 'accounts_reached']].mean()

# Display the result
sentiment_vs_engagement


KeyError: 'sentiment_category'

In [45]:
sentiment_vs_engagement = df.groupby('sentiment_category')[['plays', 'likes', 'comments', 'saves', 'accounts_reached']].mean()

KeyError: 'sentiment_category'